In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install pinecone-client==2.2.2
!pip install accelerate==0.21.0
!pip install sentence-transformers==2.2.2
!pip install huggingface-hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82




## initializing the embedding pipeline that will handle the transformation of our docs into vector embeddings.  `sentence-transformers/all-MiniLM-L6-v2`

In [ ]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

#set the device
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

#load embed model
embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
import os
import pinecone


# get API key from app.pinecone.io and environment from console
# pinecone is used to store the vectors
pinecone.init(
    api_key=os.environ.get('e9085c47-3b25-447a-95ce-e175756aa64a') or 'e9085c47-3b25-447a-95ce-e175756aa64a',
    environment=os.environ.get('gcp-starter') or 'gcp-starter'
)

In [ ]:
#build the index template
docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 2 doc embeddings, each with a dimensionality of 384.


In [ ]:
#initialize the index, create the index
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

In [ ]:
#connect to the index
index_name = 'llama-2-rag'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00152,
 'namespaces': {'': {'vector_count': 152}},
 'total_vector_count': 152}

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00152,
 'namespaces': {'': {'vector_count': 152}},
 'total_vector_count': 152}

## load model

In [ ]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # Download the model file first
  n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

# Simple inference example
output = llm(
  "<s>[INST] {prompt} [/INST]", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

# Chat Completion API

llm = Llama(model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf", chat_format="llama-2")  # Set chat_format according to the model you are using
llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ]
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto
model_id= "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = transformers.BitsAndBytesConfig(
    load_in_16bit=True
)


model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto'

)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
 ]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

NameError: name 'transformers' is not defined

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

model_name = "Deci/DeciLM-7B-instruct"

device = "cuda" # for GPU usage or "cpu" for CPU usage

quantize = True  # Optional. Useful for GPUs with less than 24GB memory

if quantize:
    dtype_kwargs = dict(quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    ))
else:
    dtype_kwargs = dict(torch_dtype="auto")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    **dtype_kwargs
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

deci_generator = pipeline("text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          temperature=0.1,
                          device_map="auto",
                          max_length=4096,
                          return_full_text=False)

system_prompt = "You are an AI assistant that follows instruction extremely well. Help as much as you can."

user_prompt = "How do I make the most delicious pancakes the world has ever tasted?"

prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
], tokenize=False, add_generation_prompt=True)

response = deci_generator(prompt)[0]['generated_text']
print(prompt + response)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

configuration_decilm.py:   0%|          | 0.00/576 [00:00<?, ?B/s]

(…)sformers_v4_35_2__configuration_llama.py:   0%|          | 0.00/9.20k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Deci/DeciLM-7B-instruct:
- transformers_v4_35_2__configuration_llama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.
### User:
How do I make the most delicious pancakes the world has ever tasted?
### Assistant:
 To make the most delicious pancakes the world has ever tasted, follow these steps:

1. Gather your ingredients:
- 1 cup all-purpose flour
- 1 cup milk
- 2 large eggs
- 2 tablespoons melted butter or oil
- 1 tablespoon sugar
- 1 teaspoon vanilla extract
- 1 teaspoon baking powder
- 1/4 teaspoon salt

2. Preheat a non-stick pan or griddle over medium heat.

3. In a mixing bowl, whisk together the eggs, milk, and melted butter or oil.

4. Add the dry ingredients (flour, sugar, baking powder, and salt) to the wet ingredients and mix until just combined. Do not overmix.

5. Grease the pan or griddle with butter or oil.

6. Pour the batter onto the pan or griddle, using a ladle or spoon to create pancakes of your desired size.

7. Cook the pancakes for 2-3 minutes on each side, or until golden brow

In [ ]:
print(prompt)

### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.
### User:
How do I make the most delicious pancakes the world has ever tasted?
### Assistant:






Used model only :


Used with LangChain :

In [ ]:
from langchain.llms import HuggingFacePipeline



## Initializing a RetrievalQA Chain

In [ ]:
from langchain.vectorstores import Pinecone

text_field = 'Question'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:73: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
query = 'What is ANGeL and how do you access it?'

vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='What are users prompted to do for accessing ANGeL?', metadata={'Answer': 'Users are prompted to register for access to ANGeL.'}),
 Document(page_content='How is ANGeL described?', metadata={'Answer': 'ANGeL is described as a web and mobile learning management system that facilitates product learning.'}),
 Document(page_content='Where can users register for ANGeL?', metadata={'Answer': 'Users can register for ANGeL at https://lift.angelms.com.my.'})]

In [ ]:
from langchain.chains import RetrievalQA


No RAG :

use RAG pipeline

use Rag and two different template


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate
from transformers import pipeline, logging


In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate
import json
import textwrap

In [ ]:
deci_generator = pipeline("text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          temperature=0.1,
                          device_map="auto",
                          max_length=4096,
                          return_full_text=False)



from transformers import pipeline, logging
# Assuming 'peft_model' and 'peft_tokenizer' are already defined and loaded

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Initialize the pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)

# Conversation history
history = " "
reference = " "
while True:
    # User inputs a question
    user_question = input("User: ")
    if user_question.lower() == 'quit':
        break
    # Assuming the result of your similarity search is stored in a variable named 'results'
    results1 = vectorstore.similarity_search(user_question, k=3)

    # Extracting the text from the metadata of the first document in the results
    text1 = str(results1[0].metadata["Answer"])
    text1 += str(results1[1].metadata["Answer"])
    text1 += str(results1[2].metadata["Answer"])
    print(text1)
    system_prompt = f"You are an AI assistant that follows instruction extremely well. Help as much as you can. Only provided the answer base on the context : {text1}"

    history +=  f"### User:\n{user_question}\n"

    prompt = f"### System:\n{system_prompt}\n{history}### Assistant:\n"

    response = deci_generator(prompt)[0]['generated_text']
    print("Chabot :"+response)

    history += f"{response}\n"

# Optionally, you can print the entire conversation at the end
print("\nFull Conversation:\n", str(history))

User: hi
Users can register for ANGeL at https://lift.angelms.com.my.Users are prompted to register for access to ANGeL.The completion of the e-learning module is required before the end of today's session to enable product learning.


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Chabot : To register for ANGeL, users can visit the website https://lift.angelms.com.my and follow the prompts to complete the registration process.
User: what is etiqa ?
Etiqa is a private limited company.Etiqa specializes in providing a range of products in both insurance and Takaful.Etiqa operates in the insurance and Takaful industry.
Chabot : Etiqa is a private limited company that specializes in providing a range of products in both insurance and Takaful. It operates in the insurance and Takaful industry.
User: i want to know more about etiqa
 More information about Etiqa can be accessed through their website at https://myetiqa.com/.The starting point is 'Mindset', which is the core belief in one's abilities to perform and complete tasks to achieve goals.Etiqa was founded 16 years ago, in 2007.
Chabot : Etiqa is a private limited company that specializes in providing a range of products in both insurance and Takaful. It operates in the insurance and Takaful industry. The company 

KeyboardInterrupt: Interrupted by user